In [66]:
import pandas as pd


fixtures_path = r'C:\Users\Sangharsh\Desktop\git\Real-Time-Match-Result-Prediction\Current_Fixtures_result.csv'
league_table_path = r'C:\Users\Sangharsh\Desktop\git\Real-Time-Match-Result-Prediction\Current_Premier_League_Table.csv'
squad_stats_path = r'C:\Users\Sangharsh\Desktop\git\Real-Time-Match-Result-Prediction\Current_Squad_Stats.csv'


fixtures_df = pd.read_csv(fixtures_path)
league_table_df = pd.read_csv(league_table_path)
squad_stats_df = pd.read_csv(squad_stats_path)


In [67]:
def parse_result(score):
    if isinstance(score, str):  
        try:
            home_score, away_score = map(int, score.split('–'))  
            if home_score > away_score:
                return 1  
            elif home_score < away_score:
                return -1  
            else:
                return 0  
        except ValueError:
            return None  
    return None  

fixtures_df['Result'] = fixtures_df['Score'].apply(parse_result)
fixtures_df_cleaned = fixtures_df[['Home', 'Away', 'xG', 'xG.1', 'Result']].dropna()



In [68]:

league_table_df = league_table_df.add_prefix('Team_')
squad_stats_df = squad_stats_df.add_prefix('Team_')

merged_data = pd.merge(
    fixtures_df_cleaned,  
    league_table_df.add_prefix('Home_'), 
    left_on='Home',  
    right_on='Home_Team_Squad', 
    how='left'
)

merged_data = pd.merge(
    merged_data,
    league_table_df.add_prefix('Away_'),  
    left_on='Away',  
    right_on='Away_Team_Squad',  
    how='left'
)

merged_data = pd.merge(
    merged_data,
    squad_stats_df.add_prefix('Home_'),  
    left_on='Home',
    right_on='Home_Team_Squad',
    how='left'
)

merged_data = pd.merge(
    merged_data,
    squad_stats_df.add_prefix('Away_'),  
    left_on='Away',
    right_on='Away_Team_Squad',
    how='left'
)



In [69]:

columns_to_drop = [
    'Home_Team_Squad_y', 'Away_Team_Squad_y', 
    'Home_Team_Squad_x', 'Away_Team_Squad_x', 
    'Home_Team_Top Team Scorer', 'Away_Team_Top Team Scorer', 
    'Home_Team_Goalkeeper', 'Away_Team_Goalkeeper',
    'Home_Team_MP_x', 'Away_Team_MP_x',  
    'Home_Team_xG_x', 'Away_Team_xG_x'   
]



merged_data_cleaned = merged_data.drop(columns=columns_to_drop, errors='ignore')




In [70]:
merged_data = merged_data_cleaned

In [71]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [72]:
def convert_results(result_str):
    result_mapping = {'W': 1, 'D': 0, 'L': -1}
    result_list = [result_mapping[char] for char in result_str.split()]
    return sum(result_list)  

merged_data['Home_Team_Last 5'] = merged_data['Home_Team_Last 5'].apply(convert_results)
merged_data['Away_Team_Last 5'] = merged_data['Away_Team_Last 5'].apply(convert_results)

In [73]:
merged_data.to_csv('merged_data.csv', index=False)

In [74]:
merged_data1 = merged_data[['Home', 'Away', 'xG', 'xG.1', 'Result', 'Home_Team_Pts/MP', 
                            'Away_Team_Pts/MP', 'Home_Team_Last 5', 'Away_Team_Last 5', 
                            'Home_Team_Rk', 'Away_Team_Rk']]

In [75]:
merged_data1.rename(columns={'xG': 'Home xG', 'xG.1': 'Away xG'}, inplace=True)


C:\Users\Sangharsh\AppData\Local\Temp\ipykernel_14384\3183478687.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data1.rename(columns={'xG': 'Home xG', 'xG.1': 'Away xG'}, inplace=True)


In [76]:
df = pd.read_html('https://fbref.com/en/comps/9/2023-2024/2023-2024-Premier-League-Stats', attrs={"id": "results2023-202491_overall"})[0]
df = df.drop(columns=['Notes'])

In [77]:
df1 = pd.read_html('https://fbref.com/en/comps/9/2023-2024/schedule/2023-2024-Premier-League-Scores-and-Fixtures', attrs={"id": "sched_2023-2024_9_1"})[0]
df1 = df1.drop(columns=['Notes'])
df1 = df1.drop(columns=['Referee'])
df1 = df1.drop(columns=['Match Report'])
df1 = df1.drop(columns=['Venue'])
df1 = df1.drop(columns=['Attendance'])

In [78]:
df1 = df1.dropna(subset=['Day'])

In [79]:
fixtures_df = df1

In [80]:
league_table_df = df

In [81]:
def parse_result(score):
    if isinstance(score, str):  
        try:
            home_score, away_score = map(int, score.split('–')) 
            if home_score > away_score:
                return 1  
            elif home_score < away_score:
                return -1  
            else:
                return 0 
        except ValueError:
            return None  
    return None  

fixtures_df['Result'] = fixtures_df['Score'].apply(parse_result)
fixtures_df_cleaned = fixtures_df[['Home', 'Away', 'xG', 'xG.1', 'Result']].dropna()


In [82]:
import pandas as pd

merged_home_df = pd.merge(
    fixtures_df_cleaned, 
    league_table_df[['Squad', 'Rk', 'Pts/MP']], 
    left_on='Home', 
    right_on='Squad', 
    how='left'
).rename(columns={'Rk': 'Home_Rk', 'Pts/MP': 'Home_Pts_MP'}).drop(columns='Squad')


final_merged_df = pd.merge(
    merged_home_df, 
    league_table_df[['Squad', 'Rk', 'Pts/MP']], 
    left_on='Away', 
    right_on='Squad', 
    how='left'
).rename(columns={'Rk': 'Away_Rk', 'Pts/MP': 'Away_Pts_MP'}).drop(columns='Squad')




In [83]:
merged_data1 = merged_data1.drop(columns=['Home_Team_Last 5'])
merged_data1 = merged_data1.drop(columns=['Away_Team_Last 5'])

In [84]:
final_merged_df.rename(columns={'xG': 'Home xG', 'xG.1': 'Away xG'}, inplace=True)
final_merged_df.rename(columns={'Home_Rk': 'Home_Team_Rk', 'Away_Rk': 'Away_Team_Rk', 'Home_Pts_MP': 'Home_Team_Pts/MP', 'Away_Pts_MP': 'Away_Team_Pts/MP'}, inplace=True)

In [85]:
combined_df = pd.concat([merged_data1, final_merged_df], ignore_index=True)

In [86]:
combined_df['Rank Diff'] = combined_df['Home_Team_Rk'] - combined_df['Away_Team_Rk']

In [87]:
combined_df['Pts/MP Diff'] = combined_df['Home_Team_Pts/MP'] - combined_df['Away_Team_Pts/MP']

In [88]:
combined_df['xG Diff'] = combined_df['Home xG'] - merged_data1['Away xG']

In [89]:
combined_df.to_csv('combined.csv', index=False)

In [90]:
df = combined_df

In [91]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

label_encoder_home = LabelEncoder()
label_encoder_away = LabelEncoder()

df['Home_encoded'] = label_encoder_home.fit_transform(df['Home'])
df['Away_encoded'] = label_encoder_away.fit_transform(df['Away'])

X = df[['Home_encoded', 'Away_encoded', 'Home xG', 'Away xG', 'Home_Team_Pts/MP', 'Away_Team_Pts/MP', 
         'Home_Team_Rk', 'Away_Team_Rk', 'Rank Diff', 
        'Pts/MP Diff', 'xG Diff']]

y = df['Result']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape


((344, 11), (86, 11), (344,), (86,))

In [92]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


y_pred = rf_model.predict(X_test)
y_proba = rf_model.predict_proba(X_test)


accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")



Accuracy: 0.5930232558139535


In [ ]:

def predict_match_result(home_team, away_team, model, df):
   
    home_encoded = label_encoder_home.transform([home_team])[0]
    away_encoded = label_encoder_away.transform([away_team])[0]
    home_xg = df.loc[df['Home'] == home_team, 'Home xG'].values[0]
    away_xg = df.loc[df['Away'] == away_team, 'Away xG'].values[0]
    home_pts_mp = df.loc[df['Home'] == home_team, 'Home_Team_Pts/MP'].values[0]  
    away_pts_mp = df.loc[df['Away'] == away_team, 'Away_Team_Pts/MP'].values[0]  
    home_rank = df.loc[df['Home'] == home_team, 'Home_Team_Rk'].values[0]
    away_rank = df.loc[df['Away'] == away_team, 'Away_Team_Rk'].values[0]
    rank_diff = df.loc[df['Home'] == home_team, 'Rank Diff'].values[0]
    pts_diff = df.loc[df['Home'] == home_team, 'Pts/MP Diff'].values[0]
    xg_diff = df.loc[df['Home'] == home_team, 'xG Diff'].values[0]
    
 
    input_data = [[home_encoded, away_encoded, home_xg, away_xg, home_pts_mp, away_pts_mp, 
                    home_rank, away_rank, rank_diff, pts_diff, xg_diff]]
    

    probabilities = model.predict_proba(input_data)[0]
    
    result = {
        'Home Win': probabilities[2],
        'Draw': probabilities[1],
        'Away Win': probabilities[0]
    }
    
    return result


In [94]:
df2 = pd.read_csv(r"C:\Users\Sangharsh\Desktop\git\Real-Time-Match-Result-Prediction\Current_Fixtures_result.csv")

In [95]:
nan_rows = df2[df2['Score'].isna()][['Home', 'Away']].head(10)

In [96]:
for index, row in nan_rows.iterrows():
    home_team = row['Home']
    away_team = row['Away']
    result = predict_match_result(home_team, away_team, rf_model, df)
    
    print(f"Match: {home_team} vs {away_team}")
    print(f"Predicted Result: {result}")
    print("-" * 40) 

Match: Newcastle Utd vs Manchester City
Predicted Result: {'Home Win': 0.3990889867148072, 'Draw': 0.29037711111643627, 'Away Win': 0.3105339021687564}
----------------------------------------
Match: Chelsea vs Brighton
Predicted Result: {'Home Win': 0.28410922286220747, 'Draw': 0.25680790319485797, 'Away Win': 0.4590828739429346}
----------------------------------------
Match: Arsenal vs Leicester City
Predicted Result: {'Home Win': 0.6783013725188397, 'Draw': 0.18981118949993356, 'Away Win': 0.1318874379812266}
----------------------------------------
Match: Brentford vs West Ham
Predicted Result: {'Home Win': 0.49287645054685014, 'Draw': 0.2659454693103344, 'Away Win': 0.2411780801428152}
----------------------------------------
Match: Everton vs Crystal Palace
Predicted Result: {'Home Win': 0.1697671356320787, 'Draw': 0.18871403879213908, 'Away Win': 0.6415188255757822}
----------------------------------------
Match: Nott'ham Forest vs Fulham
Predicted Result: {'Home Win': 0.489014

c:\Users\Sangharsh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\Sangharsh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\Sangharsh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\Sangharsh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\Sangharsh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X